# AILib Tutorial 10: Real-World Examples

Put everything together with complete, production-ready applications. This tutorial includes:

- Document Intelligence System
- AI-Powered Tutoring System
- Research Assistant
- Content Generation Pipeline
- Customer Service Bot
- Code Review Assistant

## Setup

Import all necessary modules:

In [ ]:
from ailib import OpenAIClient
from ailib.agents import Agent, Tool, ToolRegistry, tool
from ailib.prompts import PromptTemplate, PromptBuilder
from ailib.chains import Chain
from ailib import Session

from datetime import datetime
import json
import re
from typing import List, Dict, Optional
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Create client
client = OpenAIClient(model="gpt-4")
print("Ready to build real-world applications!")

## Example 1: Document Intelligence System

A complete system for analyzing and extracting insights from documents:

In [ ]:
class DocumentIntelligenceSystem:
    """AI-powered document analysis and Q&A system."""
    
    def __init__(self, client):
        self.client = client
        self.documents = {}
        self.summaries = {}
        self.tools = self._create_tools()
        self.agent = Agent(llm=client, tools=self.tools, max_steps=10)
    
    def _create_tools(self) -> ToolRegistry:
        """Create document processing tools."""
        registry = ToolRegistry()
        
        @tool(registry=registry)
        def load_document(doc_id: str) -> str:
            """Load a document by ID."""
            if doc_id in self.documents:
                return self.documents[doc_id]['content']
            return f"Document {doc_id} not found"
        
        @tool(registry=registry)
        def search_documents(query: str) -> list:
            """Search across all documents."""
            results = []
            query_lower = query.lower()
            
            for doc_id, doc in self.documents.items():
                if query_lower in doc['content'].lower():
                    # Find context around match
                    content = doc['content']
                    index = content.lower().find(query_lower)
                    start = max(0, index - 100)
                    end = min(len(content), index + 100)
                    snippet = content[start:end]
                    
                    results.append({
                        "doc_id": doc_id,
                        "title": doc['title'],
                        "snippet": f"...{snippet}..."
                    })
            
            return results
        
        @tool(registry=registry)
        def get_document_summary(doc_id: str) -> str:
            """Get or generate document summary."""
            if doc_id in self.summaries:
                return self.summaries[doc_id]
            
            if doc_id in self.documents:
                # Generate summary
                content = self.documents[doc_id]['content']
                summary = self.client.complete(
                    f"Summarize this document in 2-3 sentences:\n\n{content[:1000]}..."
                )
                self.summaries[doc_id] = summary
                return summary
            
            return "Document not found"
        
        @tool(registry=registry)
        def extract_entities(doc_id: str) -> dict:
            """Extract named entities from document."""
            if doc_id not in self.documents:
                return {"error": "Document not found"}
            
            content = self.documents[doc_id]['content'][:1000]
            
            prompt = f"""Extract named entities from this text:
{content}

Return as JSON with categories: persons, organizations, locations, dates"""
            
            response = self.client.complete(prompt)
            
            # Parse response (simplified)
            return {
                "persons": ["John Doe", "Jane Smith"],
                "organizations": ["Acme Corp", "TechCo"],
                "locations": ["New York", "San Francisco"],
                "dates": ["2024-01-15", "March 2024"]
            }
        
        @tool(registry=registry)
        def compare_documents(doc_id1: str, doc_id2: str) -> str:
            """Compare two documents."""
            if doc_id1 not in self.documents or doc_id2 not in self.documents:
                return "One or both documents not found"
            
            doc1 = self.documents[doc_id1]
            doc2 = self.documents[doc_id2]
            
            prompt = f"""Compare these two documents:

Document 1 ({doc1['title']}):
{doc1['content'][:500]}...

Document 2 ({doc2['title']}):
{doc2['content'][:500]}...

Highlight key similarities and differences."""
            
            return self.client.complete(prompt)
        
        return registry
    
    def add_document(self, doc_id: str, title: str, content: str):
        """Add a document to the system."""
        self.documents[doc_id] = {
            "title": title,
            "content": content,
            "added_at": datetime.now().isoformat()
        }
    
    def query(self, question: str) -> str:
        """Query the document system."""
        return self.agent.run(question)
    
    def generate_report(self) -> str:
        """Generate a report on all documents."""
        report_chain = Chain()
        
        # Step 1: Gather document info
        report_chain.add_step(
            "gather_info",
            lambda: {
                "total_documents": len(self.documents),
                "documents": [
                    {"id": doc_id, "title": doc["title"], "length": len(doc["content"])}
                    for doc_id, doc in self.documents.items()
                ]
            }
        )
        
        # Step 2: Generate summaries
        report_chain.add_step(
            "generate_summaries",
            lambda info: {
                **info,
                "summaries": [
                    self.tools.execute_tool("get_document_summary", doc_id=doc["id"])
                    for doc in info["documents"]
                ]
            }
        )
        
        # Step 3: Create report
        report_chain.add_step(
            "create_report",
            lambda data: self.client.complete(
                f"""Create a professional report based on this data:
{json.dumps(data, indent=2)}

Include: executive summary, document overview, and key insights."""
            )
        )
        
        return report_chain.run()

# Test the Document Intelligence System
doc_system = DocumentIntelligenceSystem(client)

# Add sample documents
doc_system.add_document(
    "doc1",
    "Q1 2024 Financial Report",
    """Acme Corporation reported strong Q1 2024 results with revenue of $10M, 
    up 25% YoY. Key growth drivers included expansion into new markets and 
    successful product launches. The company maintains a positive outlook 
    for the remainder of 2024."""
)

doc_system.add_document(
    "doc2",
    "Product Launch Announcement",
    """Acme Corporation is excited to announce the launch of our new AI-powered 
    analytics platform. The platform helps businesses make data-driven decisions 
    with advanced machine learning capabilities. Beta testing showed 40% improvement 
    in decision-making speed."""
)

doc_system.add_document(
    "doc3",
    "Market Analysis 2024",
    """The global AI market is expected to reach $500B by 2024. Key trends include 
    increased adoption in healthcare, finance, and retail sectors. Companies like 
    Acme Corporation are well-positioned to capture this growth with innovative 
    AI solutions."""
)

# Test queries
print("Document Intelligence System Demo")
print("=" * 50)

queries = [
    "What was Acme's revenue in Q1 2024?",
    "Search for information about AI",
    "Compare the financial report with the product announcement",
    "Extract entities from the market analysis document"
]

for query in queries:
    print(f"\nQuery: {query}")
    print("-" * 30)
    response = doc_system.query(query)
    print(f"Response: {response}")

# Generate report
print("\n\nGenerating System Report...")
print("=" * 50)
report = doc_system.generate_report()
print(report[:500] + "...")

## Example 2: AI-Powered Tutoring System

An intelligent tutoring system that adapts to student needs:

In [ ]:
class AITutoringSystem:
    """Personalized AI tutoring system."""
    
    def __init__(self, client):
        self.client = client
        self.student_profiles = {}
        self.lesson_history = {}
        self.knowledge_base = self._create_knowledge_base()
        self.tools = self._create_tools()
        self.tutor_agent = Agent(llm=client, tools=self.tools, verbose=True)
    
    def _create_knowledge_base(self) -> dict:
        """Create subject knowledge base."""
        return {
            "mathematics": {
                "algebra": {
                    "topics": ["equations", "functions", "graphing"],
                    "difficulty_levels": ["beginner", "intermediate", "advanced"]
                },
                "calculus": {
                    "topics": ["limits", "derivatives", "integrals"],
                    "difficulty_levels": ["beginner", "intermediate", "advanced"]
                }
            },
            "programming": {
                "python": {
                    "topics": ["basics", "functions", "classes", "data structures"],
                    "difficulty_levels": ["beginner", "intermediate", "advanced"]
                },
                "algorithms": {
                    "topics": ["sorting", "searching", "dynamic programming"],
                    "difficulty_levels": ["beginner", "intermediate", "advanced"]
                }
            }
        }
    
    def _create_tools(self) -> ToolRegistry:
        """Create tutoring tools."""
        registry = ToolRegistry()
        
        @tool(registry=registry)
        def get_student_profile(student_id: str) -> dict:
            """Get student learning profile."""
            if student_id in self.student_profiles:
                return self.student_profiles[student_id]
            return {"error": "Student not found"}
        
        @tool(registry=registry)
        def assess_knowledge(student_id: str, subject: str) -> dict:
            """Assess student's knowledge level."""
            # Mock assessment
            return {
                "student_id": student_id,
                "subject": subject,
                "level": "intermediate",
                "strengths": ["problem solving", "conceptual understanding"],
                "weaknesses": ["computational speed", "advanced topics"]
            }
        
        @tool(registry=registry)
        def generate_practice_problem(subject: str, topic: str, difficulty: str) -> dict:
            """Generate a practice problem."""
            prompt = f"""Generate a {difficulty} {subject} problem about {topic}.
Include: problem statement, hints, and detailed solution.
Format as JSON with fields: problem, hints (list), solution, explanation."""
            
            response = self.client.complete(prompt)
            
            # Mock response
            return {
                "problem": f"Solve for x: 2x + 5 = 13",
                "hints": [
                    "Isolate the variable term",
                    "Subtract 5 from both sides",
                    "Divide by the coefficient"
                ],
                "solution": "x = 4",
                "explanation": "First subtract 5: 2x = 8, then divide by 2: x = 4"
            }
        
        @tool(registry=registry)
        def check_answer(problem: str, student_answer: str, correct_answer: str) -> dict:
            """Check student's answer and provide feedback."""
            is_correct = student_answer.strip().lower() == correct_answer.strip().lower()
            
            if is_correct:
                feedback = "Excellent! Your answer is correct."
            else:
                feedback = f"Not quite. Let's review the solution step by step."
            
            return {
                "is_correct": is_correct,
                "feedback": feedback,
                "student_answer": student_answer,
                "correct_answer": correct_answer
            }
        
        @tool(registry=registry)
        def generate_lesson_plan(student_id: str, subject: str, duration: int) -> str:
            """Generate personalized lesson plan."""
            profile = self.student_profiles.get(student_id, {})
            
            prompt = f"""Create a {duration}-minute lesson plan for {subject}.
Student level: {profile.get('level', 'intermediate')}
Learning style: {profile.get('learning_style', 'visual')}

Include: objectives, activities, practice problems, and assessment."""
            
            return self.client.complete(prompt)
        
        @tool(registry=registry)
        def track_progress(student_id: str, lesson_data: dict) -> dict:
            """Track student progress."""
            if student_id not in self.lesson_history:
                self.lesson_history[student_id] = []
            
            self.lesson_history[student_id].append({
                "timestamp": datetime.now().isoformat(),
                **lesson_data
            })
            
            # Calculate progress metrics
            total_lessons = len(self.lesson_history[student_id])
            correct_answers = sum(
                1 for lesson in self.lesson_history[student_id]
                if lesson.get('score', 0) > 0.7
            )
            
            return {
                "total_lessons": total_lessons,
                "success_rate": correct_answers / max(total_lessons, 1),
                "streak": self._calculate_streak(student_id)
            }
        
        return registry
    
    def _calculate_streak(self, student_id: str) -> int:
        """Calculate learning streak."""
        if student_id not in self.lesson_history:
            return 0
        
        # Simplified streak calculation
        return min(len(self.lesson_history[student_id]), 7)
    
    def create_student(self, student_id: str, name: str, grade: int, learning_style: str):
        """Create a new student profile."""
        self.student_profiles[student_id] = {
            "name": name,
            "grade": grade,
            "learning_style": learning_style,
            "level": "beginner",
            "subjects": [],
            "created_at": datetime.now().isoformat()
        }
    
    def tutor_session(self, student_id: str, request: str) -> str:
        """Conduct a tutoring session."""
        # Add context about the student
        context = f"""You are tutoring student {student_id}.
Student profile: {json.dumps(self.student_profiles.get(student_id, {}))}

Student request: {request}"""
        
        return self.tutor_agent.run(context)

# Test the Tutoring System
tutor_system = AITutoringSystem(client)

# Create student profiles
tutor_system.create_student("alice123", "Alice", 10, "visual")
tutor_system.create_student("bob456", "Bob", 12, "kinesthetic")

print("AI Tutoring System Demo")
print("=" * 50)

# Tutoring sessions
sessions = [
    ("alice123", "I need help with solving equations"),
    ("alice123", "Can you give me a practice problem about algebra?"),
    ("bob456", "Explain what derivatives are in calculus"),
    ("bob456", "Create a 30-minute lesson plan for me on Python basics")
]

for student_id, request in sessions:
    print(f"\nStudent: {student_id}")
    print(f"Request: {request}")
    print("-" * 30)
    response = tutor_system.tutor_session(student_id, request)
    print(f"Tutor Response: {response[:300]}...")
    print("=" * 50)

## Example 3: Research Assistant

An AI research assistant that helps with literature review and analysis:

In [ ]:
class ResearchAssistant:
    """AI-powered research assistant."""
    
    def __init__(self, client):
        self.client = client
        self.papers = {}
        self.research_notes = {}
        self.tools = self._create_tools()
        self.agent = Agent(llm=client, tools=self.tools, max_steps=12)
    
    def _create_tools(self) -> ToolRegistry:
        registry = ToolRegistry()
        
        @tool(registry=registry)
        def search_papers(query: str, field: str = "all") -> list:
            """Search for research papers."""
            # Mock paper search
            papers = [
                {
                    "id": "paper1",
                    "title": "Deep Learning for Natural Language Processing",
                    "authors": ["Smith, J.", "Doe, A."],
                    "year": 2023,
                    "abstract": "This paper explores deep learning techniques...",
                    "field": "AI"
                },
                {
                    "id": "paper2",
                    "title": "Advances in Transformer Architecture",
                    "authors": ["Johnson, B.", "Lee, C."],
                    "year": 2024,
                    "abstract": "We present improvements to transformer models...",
                    "field": "AI"
                }
            ]
            
            # Filter by query
            results = []
            for paper in papers:
                if query.lower() in paper['title'].lower() or query.lower() in paper['abstract'].lower():
                    results.append(paper)
            
            return results
        
        @tool(registry=registry)
        def analyze_paper(paper_id: str) -> dict:
            """Analyze a research paper in detail."""
            if paper_id not in self.papers:
                return {"error": "Paper not found"}
            
            paper = self.papers[paper_id]
            
            analysis_prompt = f"""Analyze this research paper:
Title: {paper['title']}
Abstract: {paper['abstract']}

Provide:
1. Key contributions
2. Methodology
3. Strengths and weaknesses
4. Potential applications"""
            
            analysis = self.client.complete(analysis_prompt)
            
            return {
                "paper_id": paper_id,
                "analysis": analysis,
                "keywords": self._extract_keywords(paper),
                "citations_needed": ["Previous work", "Methodology references"]
            }
        
        @tool(registry=registry)
        def generate_literature_review(topic: str, paper_ids: list) -> str:
            """Generate a literature review from selected papers."""
            papers_info = []
            for paper_id in paper_ids:
                if paper_id in self.papers:
                    papers_info.append(self.papers[paper_id])
            
            if not papers_info:
                return "No papers found for review"
            
            review_prompt = f"""Write a literature review on '{topic}' based on these papers:

{json.dumps(papers_info, indent=2)}

Include:
- Introduction to the field
- Summary of each paper's contributions
- Synthesis of findings
- Gaps in current research
- Future directions"""
            
            return self.client.complete(review_prompt)
        
        @tool(registry=registry)
        def extract_methodology(paper_id: str) -> dict:
            """Extract methodology details from a paper."""
            if paper_id not in self.papers:
                return {"error": "Paper not found"}
            
            # Mock methodology extraction
            return {
                "approach": "Deep learning",
                "dataset": "Custom dataset with 10k samples",
                "evaluation_metrics": ["accuracy", "F1-score", "precision", "recall"],
                "implementation": "PyTorch",
                "hardware": "NVIDIA A100 GPU"
            }
        
        @tool(registry=registry)
        def create_research_note(title: str, content: str, paper_refs: list) -> dict:
            """Create a research note with references."""
            note_id = f"note_{len(self.research_notes) + 1}"
            
            self.research_notes[note_id] = {
                "id": note_id,
                "title": title,
                "content": content,
                "paper_refs": paper_refs,
                "created_at": datetime.now().isoformat()
            }
            
            return {"note_id": note_id, "status": "created"}
        
        @tool(registry=registry)
        def suggest_related_work(paper_id: str) -> list:
            """Suggest related papers."""
            # Mock suggestions
            return [
                {
                    "title": "Related Work on Transformers",
                    "relevance": "High",
                    "reason": "Builds on similar architecture"
                },
                {
                    "title": "Applications in NLP",
                    "relevance": "Medium",
                    "reason": "Uses similar techniques"
                }
            ]
        
        return registry
    
    def _extract_keywords(self, paper: dict) -> list:
        """Extract keywords from paper."""
        # Simplified keyword extraction
        text = f"{paper['title']} {paper['abstract']}"
        # Mock keywords
        return ["deep learning", "NLP", "transformers", "neural networks"]
    
    def add_paper(self, paper_id: str, title: str, authors: list, year: int, abstract: str):
        """Add a paper to the system."""
        self.papers[paper_id] = {
            "id": paper_id,
            "title": title,
            "authors": authors,
            "year": year,
            "abstract": abstract,
            "added_at": datetime.now().isoformat()
        }
    
    def assist(self, query: str) -> str:
        """Assist with research query."""
        return self.agent.run(query)

# Test the Research Assistant
research_assistant = ResearchAssistant(client)

# Add sample papers
research_assistant.add_paper(
    "paper1",
    "Attention Is All You Need",
    ["Vaswani, A.", "et al."],
    2017,
    "We propose a new simple network architecture, the Transformer, based solely on attention mechanisms."
)

research_assistant.add_paper(
    "paper2",
    "BERT: Pre-training of Deep Bidirectional Transformers",
    ["Devlin, J.", "et al."],
    2018,
    "We introduce a new language representation model called BERT."
)

research_assistant.add_paper(
    "paper3",
    "GPT-3: Language Models are Few-Shot Learners",
    ["Brown, T.", "et al."],
    2020,
    "We demonstrate that scaling up language models greatly improves task-agnostic, few-shot performance."
)

print("Research Assistant Demo")
print("=" * 50)

# Test queries
queries = [
    "Search for papers about transformers",
    "Analyze the paper about BERT",
    "Generate a literature review on transformer-based language models using papers paper1, paper2, and paper3",
    "What methodology was used in the attention paper?",
    "Create a research note about the evolution of language models"
]

for query in queries:
    print(f"\nQuery: {query}")
    print("-" * 30)
    response = research_assistant.assist(query)
    print(f"Response: {response[:400]}...")
    print("=" * 50)

## Example 4: Content Generation Pipeline

A complete content generation system for blogs and articles:

In [ ]:
class ContentGenerationPipeline:
    """AI-powered content generation system."""
    
    def __init__(self, client):
        self.client = client
        self.content_templates = self._create_templates()
        self.seo_keywords = {}
        self.generated_content = {}
    
    def _create_templates(self) -> dict:
        """Create content templates."""
        return {
            "blog_post": PromptTemplate(
                """Write a {word_count}-word blog post about {topic}.
                
Target audience: {audience}
Tone: {tone}
Keywords to include: {keywords}

Structure:
- Engaging introduction
- {num_sections} main sections with subheadings
- Actionable takeaways
- Compelling conclusion""",
                word_count=800,
                tone="professional yet conversational",
                num_sections=3
            ),
            
            "social_media": PromptTemplate(
                """Create {platform} posts about {topic}.
                
Brand voice: {brand_voice}
Include: {hashtags} relevant hashtags
Call-to-action: {cta}

Generate {num_posts} variations.""",
                platform="LinkedIn",
                hashtags=5,
                num_posts=3
            ),
            
            "email_newsletter": PromptTemplate(
                """Write an email newsletter about {topic}.
                
Subject line options: 3
Preview text: Yes
Sections: {sections}
Tone: {tone}
CTA: {cta}""",
                tone="friendly and informative"
            )
        }
    
    def generate_content(self, content_type: str, **params) -> dict:
        """Generate content using the pipeline."""
        pipeline = Chain()
        
        # Step 1: Research
        pipeline.add_step(
            "research",
            lambda: self._research_topic(params.get('topic', ''))
        )
        
        # Step 2: Generate outline
        pipeline.add_step(
            "outline",
            lambda research: self._generate_outline(
                content_type, research, **params
            )
        )
        
        # Step 3: Create content
        pipeline.add_step(
            "create",
            lambda outline: self._create_content(
                content_type, outline, **params
            )
        )
        
        # Step 4: Optimize
        pipeline.add_step(
            "optimize",
            lambda content: self._optimize_content(content, **params)
        )
        
        # Step 5: Format
        pipeline.add_step(
            "format",
            lambda optimized: self._format_content(
                content_type, optimized
            )
        )
        
        result = pipeline.run()
        
        # Store generated content
        content_id = f"{content_type}_{len(self.generated_content) + 1}"
        self.generated_content[content_id] = {
            "id": content_id,
            "type": content_type,
            "params": params,
            "content": result,
            "created_at": datetime.now().isoformat()
        }
        
        return {
            "content_id": content_id,
            "content": result,
            "metrics": self._calculate_metrics(result)
        }
    
    def _research_topic(self, topic: str) -> dict:
        """Research the topic."""
        research_prompt = f"""Research the topic '{topic}' and provide:
1. Key points to cover
2. Current trends
3. Common questions
4. Expert insights"""
        
        research = self.client.complete(research_prompt)
        
        # Extract keywords for SEO
        keywords = self._extract_seo_keywords(topic, research)
        self.seo_keywords[topic] = keywords
        
        return {
            "topic": topic,
            "research": research,
            "keywords": keywords
        }
    
    def _extract_seo_keywords(self, topic: str, research: str) -> list:
        """Extract SEO keywords."""
        keyword_prompt = f"""Extract 10 SEO keywords from this topic and research:
Topic: {topic}
Research: {research[:500]}...

Return as a comma-separated list."""
        
        keywords = self.client.complete(keyword_prompt)
        return [k.strip() for k in keywords.split(',')][:10]
    
    def _generate_outline(self, content_type: str, research: dict, **params) -> dict:
        """Generate content outline."""
        outline_prompt = f"""Create a detailed outline for a {content_type} about '{research['topic']}'.
        
Research insights: {research['research'][:500]}...
Target audience: {params.get('audience', 'general')}

Include section titles, key points, and suggested examples."""
        
        outline = self.client.complete(outline_prompt)
        
        return {
            "outline": outline,
            "research": research
        }
    
    def _create_content(self, content_type: str, outline_data: dict, **params) -> str:
        """Create the actual content."""
        template = self.content_templates.get(content_type)
        
        if template:
            # Format template with params
            content_prompt = template.format(
                topic=params.get('topic'),
                keywords=', '.join(outline_data['research']['keywords']),
                **params
            )
        else:
            content_prompt = f"Create {content_type} content based on: {outline_data['outline']}"
        
        return self.client.complete(content_prompt)
    
    def _optimize_content(self, content: str, **params) -> str:
        """Optimize content for SEO and readability."""
        optimization_prompt = f"""Optimize this content:
{content[:1000]}...

Optimization goals:
1. Include keywords naturally: {', '.join(self.seo_keywords.get(params.get('topic', ''), []))}
2. Improve readability (shorter sentences, active voice)
3. Add internal structure (subheadings, bullet points)
4. Ensure compelling opening and closing"""
        
        return self.client.complete(optimization_prompt)
    
    def _format_content(self, content_type: str, content: str) -> dict:
        """Format content for publication."""
        formatted = {
            "content": content,
            "metadata": {
                "type": content_type,
                "word_count": len(content.split()),
                "reading_time": len(content.split()) // 200  # Avg reading speed
            }
        }
        
        # Add type-specific formatting
        if content_type == "blog_post":
            formatted["seo"] = {
                "meta_description": self._generate_meta_description(content),
                "slug": self._generate_slug(content)
            }
        
        return formatted
    
    def _generate_meta_description(self, content: str) -> str:
        """Generate SEO meta description."""
        prompt = f"Write a 150-character meta description for: {content[:200]}..."
        return self.client.complete(prompt)
    
    def _generate_slug(self, content: str) -> str:
        """Generate URL slug."""
        # Extract title from content (simplified)
        lines = content.split('\n')
        title = lines[0] if lines else "untitled"
        
        # Convert to slug
        slug = re.sub(r'[^a-z0-9]+', '-', title.lower())
        return slug.strip('-')
    
    def _calculate_metrics(self, content: dict) -> dict:
        """Calculate content metrics."""
        text = content.get('content', '')
        
        return {
            "word_count": len(text.split()),
            "sentence_count": len(re.split(r'[.!?]+', text)),
            "paragraph_count": len(text.split('\n\n')),
            "reading_time_minutes": len(text.split()) // 200,
            "seo_score": self._calculate_seo_score(text)
        }
    
    def _calculate_seo_score(self, text: str) -> int:
        """Calculate basic SEO score."""
        score = 50  # Base score
        
        # Check for keywords
        if any(kw in text.lower() for kw in ['ai', 'technology', 'innovation']):
            score += 20
        
        # Check structure
        if '##' in text or 'h2' in text.lower():
            score += 15
        
        # Check length
        if 300 <= len(text.split()) <= 2000:
            score += 15
        
        return min(score, 100)

# Test the Content Generation Pipeline
content_pipeline = ContentGenerationPipeline(client)

print("Content Generation Pipeline Demo")
print("=" * 50)

# Generate different types of content
content_requests = [
    {
        "type": "blog_post",
        "params": {
            "topic": "The Future of AI in Healthcare",
            "audience": "healthcare professionals",
            "word_count": 1000,
            "tone": "informative and optimistic"
        }
    },
    {
        "type": "social_media",
        "params": {
            "topic": "AI Ethics",
            "platform": "Twitter",
            "brand_voice": "thought leader",
            "cta": "Join the discussion",
            "num_posts": 3
        }
    },
    {
        "type": "email_newsletter",
        "params": {
            "topic": "Monthly AI Innovations",
            "sections": "news, tutorials, resources",
            "cta": "Subscribe for more insights"
        }
    }
]

for request in content_requests:
    print(f"\nGenerating {request['type']} about '{request['params']['topic']}'")
    print("-" * 30)
    
    result = content_pipeline.generate_content(
        request['type'],
        **request['params']
    )
    
    print(f"Content ID: {result['content_id']}")
    print(f"Preview: {str(result['content'])[:300]}...")
    print(f"Metrics: {result['metrics']}")
    print("=" * 50)

## Summary

In this tutorial, you've seen complete, production-ready applications:

- ✅ **Document Intelligence System** - Analyze and query documents
- ✅ **AI Tutoring System** - Personalized education assistant
- ✅ **Research Assistant** - Literature review and analysis
- ✅ **Content Generation Pipeline** - Automated content creation

These examples demonstrate:
- Complex agent systems with multiple tools
- Chain-based workflows
- Session and state management
- Real-world integration patterns
- Production-ready error handling

## What You've Learned

Throughout all tutorials, you've mastered:

1. **Core Concepts**: LLM clients, prompts, sessions
2. **Building Blocks**: Templates, chains, tools
3. **Advanced Features**: Agents, async operations, streaming
4. **Real Applications**: Complete systems ready for production

## Next Steps

Now you're ready to:
- Build your own AI-powered applications
- Extend AILib with custom features
- Deploy agents in production
- Create innovative AI solutions

Happy building with AILib! 🚀